# About

Alle Titel aus der Bibliografie von Leo Konle werden in die SRU-Katalogsuche der DNB gegeben. Dabei wird immer nach Titel und Verlag gesucht. Ausgewählte Daten der Ergebnisse werden als csv gespeichert. Die vorhandenen Umfangsangaben werden ausgewertet.

In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree

# Marc-Doku
[Controlfield 007 Medientyp](https://www.loc.gov/marc/bibliographic/bd007c.html)


In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/LeKonArD/heftroman-universum/master/plot_hefte_.csv', sep='\t')
df

,genre,title,start,end,publisher,count,Quelle
0,Abenteuer,AUS ALLEN TEILEN DER WELT,1899,1899,Münchmeyer,5,BDH
1,Abenteuer,AUS ALLEN WELTTEILEN,1899,1899,Münchmeyer,5,BDH
2,unklar,MODERNE ZEHNPFENNIG BIBLIOTHEK,1901,?,Klambt,246,BDH
3,Erotik,EIN BLICK DURCH'S SCHLÜSSELLOCH,1902,1903,Dresdner,48,BDH
4,unklar,GEHEIMNISVOLLE BIBLIOTHEK,1902,1906,Dresdner,160,BDH
...,...,...,...,...,...,...,...
705,unklar,TOM HYPNOS,?,?,Bison,12,BDH
706,Western,TRAPPER TREUHERZ,?,?,MVV,12,BDH
707,unklar,WAS MAN NICHT LAUT ERZÄHLT,?,?,Mitteldeutsche Verlagsanstalt,55,BDH
708,Western,WILD-WEST-BIBLIOTHEK,?,?,Eichler,25,BDH


In [14]:
def dnb_sru(titel, publisher):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': f"tit = {titel} and vlg = {publisher}",
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

In [20]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'

    # media_type
    media_type = xml.xpath("marc:controlfield[@tag = '007']", namespaces=ns)
    try:
        media_type = media_type[0].text
    except:
        media_type = 'fail'
    
    # umfang
    umfang = xml.xpath("marc:datafield[@tag = '300']/marc:subfield[@code = 'a']", namespaces=ns)
    
    try:
        umfang = umfang[0].text
        #umfang = unicodedata.normalize("NFC", umfang)
    except:
        umfang = "unkown"

    # serien
    serie_titel = xml.xpath("marc:datafield[@tag = '490']/marc:subfield[@code = 'a']", namespaces=ns)
    serie_band = xml.xpath("marc:datafield[@tag = '490']/marc:subfield[@code = 'v']", namespaces=ns)

    try:
        serie_titel = serie_titel[0].text
    except:
        serie_titel = "unkown"

    try:
        serie_band = serie_band[0].text
    except:
        serie_band = "unkown"
        
    # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        titel = titel[0].text
    except:
        titel = "unkown"
    
    # autor
    autor = xml.xpath("marc:datafield[@tag = '100']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        autor = autor[0].text
    except:
        autor = "unkown"

    # part_number
    part_number = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'n']", namespaces=ns)
    try:
        part_number = part_number[0].text
    except:
        part_number = "unkown"
    
    # part_titel
    part_titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'p']", namespaces=ns)
    try:
        part_titel = part_titel[0].text
    except:
        part_titel = "unkown"

    # jahr
    jahr = xml.xpath("marc:datafield[@tag = '264']/marc:subfield[@code = 'c']", namespaces=ns)
    try:
        jahr = jahr[0].text
    except:
        jahr = "unkown"

    meta_dict = {"idn":idn, "media_type":media_type, "umfang":umfang, "serie_titel":serie_titel, "serie_band":serie_band, "titel":titel, "autor":autor, "part_number":part_number, "part_titel":part_titel, "jahr":jahr}
    return meta_dict

In [23]:
df_all = pd.DataFrame()

for row in df.itertuples():
    records = dnb_sru(row.title, row.publisher)
    output = [parse_record(record) for record in records]
    df_all = df_all.append(output, ignore_index=True)

df_all


,idn,media_type,umfang,serie_titel,serie_band,titel,autor,part_number,part_titel,jahr
0,1236850289,cr||||||||||||,unkown,unkown,unkown,Moderne 10-Pfennig-Bibliothek,unkown,unkown,unkown,1912-1915
1,979424100,tu,unkown,unkown,unkown,Moderne 10-Pfennig-Bibliothek,unkown,unkown,unkown,1905-1915
2,362945330,tu,62 S.,Geheimnisvolle Bibliothek,Bd. 2,Der schwarze Don Juan,"Vermont, H. de",unkown,unkown,1902
3,362059853,tu,63 S.,Geheimnisvolle Bibliothek,Bd. 11,"Die Cancantänzerin aus ""Moulin Rouge""","Pompé, Charlotte",unkown,unkown,[um 1902]
4,578018888,tu,62 S.,unkown,unkown,Geheimnisvolle Bibliothek,unkown,Bd. 1.,In verrufenem Hause / Jadwiga Grünberg,1902-
...,...,...,...,...,...,...,...,...,...,...
31392,1001871995,tu,Je 16 S.,unkown,unkown,Schlumperfritz und Schlamperfranz,unkown,Streich 21-32,unkown,[1922]
31393,560894082,tu,unkown,unkown,unkown,Schlumperfritz und Schlamperfranz,unkown,unkown,unkown,unkown
31394,1017669856,cr||||||||||||,unkown,unkown,unkown,Schlumperfritz und Schlamperfranz,unkown,unkown,unkown,unkown
31395,580196917,tu,319 S.,Sonnen-Strahlen,[1],Im Zuchthause,"Hirschfeld, Max",unkown,unkown,[1920]


In [24]:
df_all.to_csv('umfangsschaetzung.csv')

In [48]:
umfaenge = df_all[df_all['media_type'] == 'tu'].umfang.str.extract('(\d+)\sS[eiten.]+')
umfaenge.rename(columns={0:'seiten'}, inplace=True)
umfaenge.to_csv('umfangsschaetzung_umfang.csv')


In [66]:
# Durchschnitt
umfaenge[umfaenge.seiten.notna()].seiten.astype(int).mean()


99.62364386263785

In [67]:
# Median
umfaenge[umfaenge.seiten.notna()].seiten.astype(int).median()

64.0